In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
class Medicine:
    def __init__(self, updateAt, type, date, name, sheet, unit, housou):
        self.updateAt = updateAt
        self.type = type
        self.date = date
        self.name = name
        self.sheet = sheet
        self.unit = unit
        self.housou = housou

    # dataListを受け取って、Medicineクラスのインスタンスを返す
    def create(dataList):
        updateAt = list(map(lambda x: x.split('：')[0], dataList)) 
        type = list(map(lambda x: x.split('：')[1][1:], dataList)) 
        date = list(map(lambda x: x.split('：')[2].split(')')[0], dataList))
        name = list(map(lambda x: x.split('：')[2].split(')')[1].split(' ')[0], dataList))
        sheet = list(map(lambda x: x.split('：')[2].split(')')[1].split(' ')[1], dataList))
        unit = list(map(lambda x: x.split('：')[2].split(')')[1].split(' ')[2].split('\u3000')[0], dataList))
        housou = list(map(lambda x: x.split('：')[2].split(')')[1].split(' ')[2].split('\u3000')[1], dataList)) 

        length = len(dataList)

        return list(map(lambda x: Medicine(updateAt[x], type[x], date[x], name[x], sheet[x], unit[x], housou[x]), range(length)))

def convertToDF(dataList):
    return pd.DataFrame(
        data = {
            'updateAt': list(map(lambda x: x.updateAt, dataList)),
            'type': list(map(lambda x: x.type, dataList)),
            'date': list(map(lambda x: x.date, dataList)),
            'name': list(map(lambda x: x.name, dataList)),
            'sheet': list(map(lambda x: x.sheet, dataList)),
            'unit': list(map(lambda x: x.unit, dataList)),
            'housou': list(map(lambda x: x.housou, dataList))
    }
)
    

In [3]:
res = requests.get('https://drugshortage.jp/log.php')
soup = BeautifulSoup(res.text, 'html.parser')

In [4]:
textarea = soup.find('textarea')

In [5]:
dataList = textarea.string.splitlines()
dataList = dataList[3:(len(dataList)-1)] # 先頭3行と最後1行はムダデータなので削除
list(map(lambda x: x.replace('\u3000', '：'), dataList)) #unicodeのスペースを：に置き換える

['2021年11月27日：(販売中止：2021年11月26日)モンテルカストＯＤ錠１０ｍｇ「タカタ」 ＰＴＰ 14錠：420錠包装',
 '2021年11月27日：(出荷調整：2021年11月26日)ドンペリドン錠５ｍｇ「杏林」 ＰＴＰ 10錠：100錠包装',
 '2021年11月27日：(出荷調整：2021年11月26日)クラリスロマイシン錠５０ｍｇ小児用「杏林」 ＰＴＰ 10錠：100錠包装',
 '2021年11月27日：(出荷調整：2021年11月26日)ドンペリドン錠１０ｍｇ「杏林」 ＰＴＰ 10錠：100錠包装',
 '2021年11月27日：(出荷調整：2021年11月26日)ドンペリドン錠１０ｍｇ「杏林」 ＰＴＰ 10錠：1000錠包装']

In [6]:
medList = Medicine.create(dataList)

In [7]:
medList[0].name

'モンテルカストＯＤ錠１０ｍｇ「タカタ」'

In [8]:
df = convertToDF(medList)
df

,updateAt,type,date,name,sheet,unit,housou
0,2021年11月27日,販売中止,2021年11月26日,モンテルカストＯＤ錠１０ｍｇ「タカタ」,ＰＴＰ,14錠,420錠包装
1,2021年11月27日,出荷調整,2021年11月26日,ドンペリドン錠５ｍｇ「杏林」,ＰＴＰ,10錠,100錠包装
2,2021年11月27日,出荷調整,2021年11月26日,クラリスロマイシン錠５０ｍｇ小児用「杏林」,ＰＴＰ,10錠,100錠包装
3,2021年11月27日,出荷調整,2021年11月26日,ドンペリドン錠１０ｍｇ「杏林」,ＰＴＰ,10錠,100錠包装
4,2021年11月27日,出荷調整,2021年11月26日,ドンペリドン錠１０ｍｇ「杏林」,ＰＴＰ,10錠,1000錠包装


In [9]:
df.to_csv('drug.csv', index=False)